# NEXUS tool: case study for the NWSAS basin - least-cost options calculations
In this notebook a case study for the NWSAS basin is covered using the `nexus_tool` package. The energy requirements for agricultural irrigation were previously calculated (check the water demand and energy demand case studies). In this case study, the least-cost options to supply the energy needs for agricultural irrigation are identified.

First import the package by running the following block:

In [54]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [61]:
%autoreload
import nexus_tool

[autoreload of nexus_tool failed: Traceback (most recent call last):
  File "C:\Users\camilorg\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\extensions\autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "C:\Users\camilorg\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\extensions\autoreload.py", line 450, in superreload
    update_generic(old_obj, new_obj)
  File "C:\Users\camilorg\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\extensions\autoreload.py", line 387, in update_generic
    update(a, b)
  File "C:\Users\camilorg\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\extensions\autoreload.py", line 342, in update_class
    if update_generic(old_obj, new_obj): continue
  File "C:\Users\camilorg\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\extensions\autoreload.py", line 387, in update_generic
    update(a, b)
  File "C:\Users\camilorg\AppData\Local\Continuum\anaconda3\lib\site-packa

In [62]:
file_path = r'nwsas_results/pumping_energy.csv'
df = nexus_tool.read_csv(file_path)

In [63]:
nwsas = nexus_tool.Model(df)

In [64]:
nwsas.create_wind_turbine('Wind power')

In [65]:
nwsas.get_wind_cf('Wind power')

In [67]:
nwsas.technologies['Wind power'].cf

,wind_cf_1,wind_cf_2,wind_cf_3,wind_cf_4,wind_cf_5,wind_cf_6,wind_cf_7,wind_cf_8,wind_cf_9,wind_cf_10,wind_cf_11,wind_cf_12
0,0.016977,0.061987,0.052534,0.094223,0.082888,0.077431,0.057172,0.039743,0.028734,0.022432,0.010380,0.012369
1,0.016977,0.061987,0.052534,0.094223,0.082888,0.088488,0.057172,0.039743,0.039743,0.022432,0.010380,0.022432
2,0.016977,0.072123,0.052534,0.094223,0.088488,0.088488,0.057172,0.039743,0.039743,0.022432,0.010380,0.016977
3,0.016977,0.052534,0.048080,0.094223,0.088488,0.088488,0.057172,0.039743,0.028734,0.022432,0.012369,0.016977
4,0.016977,0.061987,0.052534,0.094223,0.088488,0.088488,0.057172,0.039743,0.039743,0.022432,0.010380,0.022432
5,0.016977,0.072123,0.057172,0.088488,0.088488,0.077431,0.057172,0.039743,0.028734,0.022432,0.010380,0.022432
6,0.016977,0.072123,0.057172,0.088488,0.088488,0.088488,0.057172,0.039743,0.039743,0.022432,0.010380,0.022432
7,0.019598,0.072123,0.052534,0.072123,0.088488,0.088488,0.057172,0.039743,0.039743,0.022432,0.010380,0.022432
8,0.019598,0.072123,0.052534,0.072123,0.088488,0.088488,0.057172,0.039743,0.039743,0.022432,0.010380,0.022432
9,0.016977,0.061987,0.057172,0.088488,0.088488,0.088488,0.057172,0.039743,0.039743,0.022432,0.012369,0.022432
